In [ ]:
import sqlite3
import csv

conn = sqlite3.connect("sqlite.db")
cursor = conn.cursor()

In [ ]:
# Create table
cursor.execute(
    """
CREATE TABLE IF NOT EXISTS sales (
    Item_Fat_Content TEXT,
    Item_Identifier TEXT,
    Item_Type TEXT,
    Outlet_Establishment_Year INTEGER,
    Outlet_Identifier TEXT,
    Outlet_Location_Type TEXT,
    Outlet_Size TEXT,
    Outlet_Type TEXT,
    Item_Visibility REAL,
    Item_Weight REAL,
    Sales REAL,
    Rating REAL
)
"""
)


def to_float(val):
    return float(val) if val.strip() else None  # or 0.0 if you prefer


def to_int(val):
    return int(val) if val.strip() else None  # or 0


with open("./blinkit_grocery_data.csv", newline="", encoding="utf-8-sig") as csvfile:
    reader = csv.DictReader(csvfile)
    rows = [
        (
            row["Item Fat Content"],
            row["Item Identifier"],
            row["Item Type"],
            to_int(row["Outlet Establishment Year"]),
            row["Outlet Identifier"],
            row["Outlet Location Type"],
            row["Outlet Size"],
            row["Outlet Type"],
            to_float(row["Item Visibility"]),
            to_float(row["Item Weight"]),
            to_float(row["Sales"]),
            to_float(row["Rating"]),
        )
        for row in reader
    ]


cursor.executemany(
    """
INSERT INTO sales VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""",
    rows,
)

# Commit
conn.commit()

print("CSV data imported into SQLite database successfully.")

CSV data imported into SQLite database successfully.


In [ ]:
cursor.execute(
    "SELECT item_identifier, item_weight, count(item_weight) FROM sales GROUP BY item_identifier, item_weight LIMIT 10"
)
rows = cursor.fetchall()
for row in rows:
    print(row)

('DRA12', 11.6, 6)
('DRA24', None, 0)
('DRA24', 19.35, 5)
('DRA59', None, 0)
('DRA59', 8.27, 6)
('DRB01', None, 0)
('DRB01', 7.39, 2)
('DRB13', 6.115, 5)
('DRB24', 8.785, 4)
('DRB25', None, 0)


In [ ]:
# Update item_weight for NULL values
cursor.execute(
    """
UPDATE sales
SET item_weight = (
    SELECT DISTINCT a.item_weight
    FROM sales a
    WHERE a.item_identifier = sales.item_identifier
    AND a.item_weight IS NOT NULL
)
WHERE item_weight IS NULL
"""
)
conn.commit()

In [ ]:
cursor.execute(
    "UPDATE sales SET item_fat_content = 'Low Fat' WHERE item_fat_content = 'LF'"
)
cursor.execute(
    "UPDATE sales SET item_fat_content = 'Low Fat' WHERE item_fat_content = 'low fat'"
)
cursor.execute(
    "UPDATE sales SET item_fat_content = 'Regular' WHERE item_fat_content = 'reg'"
)
conn.commit()

In [ ]:
cursor.execute(
    """SELECT a.item_identifier, 
               a.item_fat_content, 
               a.item_type, 
               a.item_weight, 
               count(a.item_identifier) 
               FROM 
               (SELECT DISTINCT item_identifier, 
               item_fat_content, 
               item_type, 
               item_weight 
               FROM sales) a 
               GROUP BY a.item_identifier 
               HAVING count(a.item_identifier) > 1"""
)

rows = cursor.fetchall()
for row in rows:
    print(row)

In [ ]:
# create an item table with item_identifier, item_fat_content, item_type, item_weight
cursor.execute(
    """
CREATE TABLE IF NOT EXISTS items (
    item_identifier TEXT PRIMARY KEY,
    item_fat_content TEXT,
    item_type TEXT,
    item_weight REAL
)
"""
)

# insert data into items table
cursor.execute(
    """
INSERT INTO items (item_identifier, item_fat_content, item_type, item_weight)
SELECT DISTINCT item_identifier, item_fat_content, item_type, item_weight FROM sales
"""
)
conn.commit()

In [ ]:
# create an outlet table with outlet_identifier, outlet_establishment_year, outlet_location_type, outlet_size, outlet_type
cursor.execute(
    """
CREATE TABLE IF NOT EXISTS outlets (
    outlet_identifier TEXT PRIMARY KEY,
    outlet_establishment_year INTEGER,
    outlet_location_type TEXT,
    outlet_type TEXT
)
"""
)

# insert data into outlets table
cursor.execute(
    """
INSERT INTO outlets (outlet_identifier, outlet_establishment_year, outlet_location_type, outlet_type)
SELECT DISTINCT outlet_identifier, outlet_establishment_year, outlet_location_type, outlet_type FROM sales
"""
)
conn.commit()

In [ ]:
# create a sales table with item_identifier, outlet_identifier, item_visibility, sales, rating
cursor.execute(
    """
CREATE TABLE sales_new AS
SELECT 
    item_identifier,
    outlet_identifier,
    outlet_size,
    item_visibility,
    sales,
    rating
FROM sales
"""
)

cursor.execute("DROP TABLE sales")

cursor.execute("ALTER TABLE sales_new RENAME TO sales")

conn.commit()

In [11]:
conn.close()